##ML + STRUCTURED STREAMING

#Costruzione di un modello ML e predizione in tempo reale sfruttando lo straming strutturato



In [43]:
#IMPORT
import pyspark.ml.regression as rg
import pyspark as pys 
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
import pyspark.sql as sql
import pyspark.sql.functions as f
import pyspark.ml.feature as feat
import numpy as np
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
import json 


import pyspark.ml.evaluation as ev

In [21]:

#Inizializzazione spark session/context e caricamento dataset

#Spark session
spark = SparkSession \
    .builder \
    .appName("StructuredNetworkWordCount") \
    .getOrCreate()

#Spark context
sc = spark.sparkContext

#Caricamento dataset da file CSV
forest_path = 'forest_coverage_type.csv'

forest=spark.read.csv(
    forest_path,
    header=True,
    inferSchema=True
)

print("Schema DF: ")
forest.printSchema()

#Divisione fra train e test
forest_train, forest_test=(
    forest.randomSplit([0.7,0.3],seed=123)
)

#Esporto il dataset di test in formato CSV (in un unico file con repartition(1))
forest_test.repartition(1).write.csv("forest_test.csv","overwrite")

Schema DF: 
root
 |-- Elevation: integer (nullable = true)
 |-- Aspect: integer (nullable = true)
 |-- Slope: integer (nullable = true)
 |-- Horizontal_Distance_To_Hydrology: integer (nullable = true)
 |-- Vertical_Distance_To_Hydrology: integer (nullable = true)
 |-- Horizontal_Distance_To_Roadways: integer (nullable = true)
 |-- Hillshade_9am: integer (nullable = true)
 |-- Hillshade_Noon: integer (nullable = true)
 |-- Hillshade_3pm: integer (nullable = true)
 |-- Horizontal_Distance_To_Fire_Points: integer (nullable = true)
 |-- Wilderness_Area_Rawah: integer (nullable = true)
 |-- Wilderness_Area_Neota: integer (nullable = true)
 |-- Wilderness_Area_Comanche: integer (nullable = true)
 |-- Wilderness_Area_CacheLaPoudre: integer (nullable = true)
 |-- Soil_type_2702: integer (nullable = true)
 |-- Soil_type_2703: integer (nullable = true)
 |-- Soil_type_2704: integer (nullable = true)
 |-- Soil_type_2705: integer (nullable = true)
 |-- Soil_type_2706: integer (nullable = true)
 |--

In [31]:
#Creazione modello di ML

#Cerchiamo di predirre la colonna Elevation che è la prima nel dataset

#Usiamo una pipeline in 2 stadi per la creazione di modelli ML in PySpark

#1) Istanzione un oggetto della classe VectorAssembler
#Che per permette di fondere tutte le colonne in una
vectorAssembler = feat.VectorAssembler(
    inputCols=forest.columns[1:] #la 1 è "elevation"
    , outputCol='features'
    )

#2) Istanzione un oggetto della classe RandomForestRegressor
#Implementa l'algoritmo di regressione "Random Forest" e crea il modello ML
rf_obj = rg.RandomForestRegressor(
    labelCol='Elevation' #target value (colonna da predire)
    , maxDepth=10
    , minInstancesPerNode=10
    , minInfoGain=0.1
    , numTrees=10
    )

#Pipeline 
pip = Pipeline(stages=[vectorAssembler, rf_obj])

#Modello ML 
pModel = ( #DF come quello di input ma con le colonne features e predicition 
    pip.fit(forest_train)
)


TypeError: save() missing 1 required positional argument: 'path'

In [50]:
#Esportazione dati

#Salvo il modello ML ottenuto
modelPath="/home/lorenzo/Documenti/PySpark-tesi/Prototipo/exported"
pModel.write().overwrite().save(modelPath)

#Esporto lo schema del dataset forest

with open(modelPath+"/schema.json", "w") as f:
    json.dump(forest.schema.jsonValue(), f)

In [13]:
#Test del modello su forest_test (solo batch)
results=(
    pModel
    .transform(forest_test)
    .select("Elevation","prediction")
)

#5 predizioni di esempio (da forest_train)
results.show(5)

+---------+------------------+
|Elevation|        prediction|
+---------+------------------+
|     1879|2133.8356236533614|
|     1896|2164.4272321820235|
|     1901| 2092.579629892129|
|     1904|1977.2831955407844|
|     1905|1976.4692019172376|
+---------+------------------+
only showing top 5 rows



In [12]:
#Valutazione accuratezza 
evaluator = ev.RegressionEvaluator(labelCol='Elevation')
evaluator.evaluate(results, {evaluator.metricName: 'r2'})

0.821924491416199

In [35]:
#Utilizzo il modello cercando di predirre "elavation" nei record ricevuti dalla socket

#Disattivo il log di livello ALL (altrimenti output illegibile)
spark.sparkContext.setLogLevel('error')

#DF che rappresenta i dati letti dalla socket
socketDF = spark\
    .readStream\
    .format('socket')\
    .option('host', 'localhost')\
    .option('port', 9999)\
    
    

#Esporto lo schema del DF di partenza: forest


